In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
import math
import matplotlib.pyplot as plt

In [2]:
str_files = ''

#df = pd.read_csv(str_files + "ph.csv",encoding='utf-8',sep=';')
dateparse = lambda dates:pd.datetime.strptime(dates,'%d/%m/%Y')
df = pd.read_csv(str_files + 'od.CSV',encoding='utf-8',sep=';', parse_dates = ['data_coleta'],index_col=['data_coleta'],date_parser=dateparse)

In [3]:
z = df.loc[df['UGRHI'] == '10 - SOROCABA/MEDIO TIETE']

In [4]:
z = z.dropna(axis=1,how='any')

In [5]:
trainingd = z.resample('3M').mean()
trainingd
data = trainingd.iloc[:,1:2].values
train = trainingd.iloc[0:132,1:2].values  
test = trainingd.iloc[132:,1:2].values   
print('treino:',len(train))
print('teste:',len(test))

treino: 132
teste: 34


In [6]:
sc = MinMaxScaler()
train = sc.fit_transform(train)
test = sc.fit_transform(test)

In [7]:
def prepare_data(data, lags):
    X, y = [], []
    for row in range(len(data) - lags - 1):
        a = data[row:(row + lags), 0]
        X.append(a)
        y.append(data[row + lags, 0])
    return np.array(X), np.array(y)  

In [8]:
lags = 1
train_X, train_y = prepare_data(train, lags)
test_X, test_y = prepare_data(test, lags)

train_X = np.reshape(train_X, (train_X.shape[0],1,train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0],1,test_X.shape[1]))
print('Novo formato de treino:')
print(train_X.shape) 

Novo formato de treino:
(130, 1, 1)


In [9]:
model = Sequential() 

model.add(LSTM(units = 10, return_sequences = True, input_shape = (train_X.shape[1], 1))) 
model.add(Dropout(0.2))
model.add(LSTM(units = 10, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 10, return_sequences = False)) 
model.add(Dropout(0.2))
model.add(Dense(units = 1)) 
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

#es = EarlyStopping(monitor='val_loss',mode='auto',verbose = 1,patience=5)

model.fit(train_X, train_y, epochs = 100, batch_size = 32, validation_data=(test_X,test_y))

Train on 130 samples, validate on 32 samples
Epoch 1/100
130/130 [==============================] - 18s 136ms/sample - loss: 0.2331 - val_loss: 0.0757
Epoch 2/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.2241 - val_loss: 0.0708
Epoch 3/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.2151 - val_loss: 0.0660
Epoch 4/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.2060 - val_loss: 0.0613
Epoch 5/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1975 - val_loss: 0.0566
Epoch 6/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1898 - val_loss: 0.0521
Epoch 7/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1790 - val_loss: 0.0477
Epoch 8/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1691 - val_loss: 0.0435
Epoch 9/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1625 - val_loss: 0.0394
Epoch 10/100
130/130 [===

130/130 [==============================] - 0s 2ms/sample - loss: 0.0383 - val_loss: 0.0419
Epoch 80/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0419 - val_loss: 0.0416
Epoch 81/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0389 - val_loss: 0.0409
Epoch 82/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0457 - val_loss: 0.0388
Epoch 83/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0403 - val_loss: 0.0381
Epoch 84/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0407 - val_loss: 0.0372
Epoch 85/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0416 - val_loss: 0.0362
Epoch 86/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0412 - val_loss: 0.0361
Epoch 87/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0396 - val_loss: 0.0365
Epoch 88/100
130/130 [==============================] - 0s 1ms/sample - loss:

In [10]:
train_score10 = model.evaluate(train_X, train_y, verbose=0)
print(train_score10)
print('Pontuação de Treino: {:.2f} MSE ({:.2f} RMSE)'.format(train_score10, math.sqrt(train_score10)))
test_score10 = model.evaluate(test_X, test_y, verbose=0)
print('Pontuação de teste: {:.2f} MSE ({:.2f} RMSE)'.format(test_score10, math.sqrt(test_score10)))

0.037102065808497946
Pontuação de Treino: 0.04 MSE (0.19 RMSE)
Pontuação de teste: 0.04 MSE (0.19 RMSE)


In [11]:
model = Sequential() 

model.add(LSTM(units = 25, return_sequences = True, input_shape = (train_X.shape[1], 1))) 
model.add(Dropout(0.2))
model.add(LSTM(units = 25, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 25, return_sequences = False)) 
model.add(Dropout(0.2))
model.add(Dense(units = 1)) 
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

#es = EarlyStopping(monitor='val_loss',mode='min',verbose = 1)

model.fit(train_X, train_y, epochs = 100, batch_size = 32, validation_data=(test_X,test_y))

Train on 130 samples, validate on 32 samples
Epoch 1/100
130/130 [==============================] - 17s 127ms/sample - loss: 0.2325 - val_loss: 0.0740
Epoch 2/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.2201 - val_loss: 0.0673
Epoch 3/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.2080 - val_loss: 0.0606
Epoch 4/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1943 - val_loss: 0.0539
Epoch 5/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1813 - val_loss: 0.0471
Epoch 6/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.1669 - val_loss: 0.0403
Epoch 7/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1536 - val_loss: 0.0338
Epoch 8/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1364 - val_loss: 0.0274
Epoch 9/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.1192 - val_loss: 0.0219
Epoch 10/100
130/130 [===

130/130 [==============================] - 0s 2ms/sample - loss: 0.0382 - val_loss: 0.0359
Epoch 80/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.0396 - val_loss: 0.0367
Epoch 81/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0375 - val_loss: 0.0358
Epoch 82/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0402 - val_loss: 0.0356
Epoch 83/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0381 - val_loss: 0.0355
Epoch 84/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0421 - val_loss: 0.0351
Epoch 85/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0398 - val_loss: 0.0374
Epoch 86/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0428 - val_loss: 0.0407
Epoch 87/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0370 - val_loss: 0.0401
Epoch 88/100
130/130 [==============================] - 0s 2ms/sample - loss:

In [12]:
train_score25 = model.evaluate(train_X, train_y, verbose=0)
print(train_score25)
print('Pontuação de Treino: {:.2f} MSE ({:.2f} RMSE)'.format(train_score25, math.sqrt(train_score25)))
test_score25 = model.evaluate(test_X, test_y, verbose=0)
print('Pontuação de teste: {:.2f} MSE ({:.2f} RMSE)'.format(test_score25, math.sqrt(test_score25)))

0.03819407178805424
Pontuação de Treino: 0.04 MSE (0.20 RMSE)
Pontuação de teste: 0.03 MSE (0.18 RMSE)


In [13]:
model = Sequential() 

model.add(LSTM(units = 50, return_sequences = True, input_shape = (train_X.shape[1], 1))) 
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = False)) 
model.add(Dropout(0.2))
model.add(Dense(units = 1)) 
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

#es = EarlyStopping(monitor='val_loss',mode='auto',verbose = 1)

model.fit(train_X, train_y, epochs = 100, batch_size = 32, validation_data=(test_X,test_y))

Train on 130 samples, validate on 32 samples
Epoch 1/100
130/130 [==============================] - 20s 154ms/sample - loss: 0.2295 - val_loss: 0.0698
Epoch 2/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.2096 - val_loss: 0.0591
Epoch 3/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1877 - val_loss: 0.0479
Epoch 4/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1641 - val_loss: 0.0367
Epoch 5/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1386 - val_loss: 0.0262
Epoch 6/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1102 - val_loss: 0.0180
Epoch 7/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0813 - val_loss: 0.0151
Epoch 8/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0561 - val_loss: 0.0216
Epoch 9/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0403 - val_loss: 0.0394
Epoch 10/100
130/130 [===

130/130 [==============================] - 0s 2ms/sample - loss: 0.0389 - val_loss: 0.0354
Epoch 80/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0393 - val_loss: 0.0378
Epoch 81/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0408 - val_loss: 0.0376
Epoch 82/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0373 - val_loss: 0.0368
Epoch 83/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0369 - val_loss: 0.0327
Epoch 84/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0406 - val_loss: 0.0295
Epoch 85/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0383 - val_loss: 0.0336
Epoch 86/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0373 - val_loss: 0.0387
Epoch 87/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0374 - val_loss: 0.0414
Epoch 88/100
130/130 [==============================] - 0s 2ms/sample - loss:

In [14]:
train_score50 = model.evaluate(train_X, train_y, verbose=0)
print(train_score50)
print('Pontuação de Treino: {:.2f} MSE ({:.2f} RMSE)'.format(train_score50, math.sqrt(train_score50)))
test_score50 = model.evaluate(test_X, test_y, verbose=0)
print('Pontuação de teste: {:.2f} MSE ({:.2f} RMSE)'.format(test_score50, math.sqrt(test_score50)))

0.03666187822818756
Pontuação de Treino: 0.04 MSE (0.19 RMSE)
Pontuação de teste: 0.04 MSE (0.20 RMSE)
